Trong tệp này, em thực hiện cài đặt chương trình sử dụng HMM (segmental K-means) để nhận dạng khẩu lệnh đơn lẻ (sử dụng HMM với Mixture of Gaussians, sử dụng toàn bộ bộ dữ liệu đã thu), sử dụng tất cả các label trừ "sil"

Để có được mô hình HMM, em sử dụng thư viện hmmlearn

# Khai báo thư viện và các đường dẫn cần thiết

In [3]:
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import fnmatch
import pandas as pd
import os.path
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
!pip install hmmlearn
import hmmlearn.hmm as hmm
from sklearn.utils import shuffle

     |████████████████████████████████| 129 kB 28.3 MB/s 


In [6]:
from google.colab import drive
drive.mount("/content/drive")
DATA_ROOT_DIR="/content/drive/MyDrive/Speech/Data_14/"
!ls $DATA_ROOT_DIR
DATA_MFCC_DIR=DATA_ROOT_DIR+"MFCC/"
!ls $DATA_MFCC_DIR

Mounted at /content/drive
A  ban	MFCC	     model.pkl		nhay  trai
B  len	model_1.pkl  model_without.pkl	phai  xuong
A_mfcc.npy    B_mfcc.npy    nhay_mfcc.npy  trai_mfcc.npy
ban_mfcc.npy  len_mfcc.npy  phai_mfcc.npy  xuong_mfcc.npy


# Xử lý dữ liệu
Trong mục này, em chuẩn bị dữ liệu để huấn luyện và đánh giá mô hình
- Load dữ liệu từ file đặc trưng MFCC đã tính toán trước đó
- Chia tập dữ liệu train và test với tỷ lệ 70:30

In [7]:
class_names = ["len", "xuong", "phai", "B", "A", "trai", "nhay", "ban"]
features = []
labels = []

#Với từng label
for cname in class_names:
  
  #Load file MFCC
  mfcc_data = np.load(DATA_MFCC_DIR+cname+"_mfcc.npy",allow_pickle=True)
  len_label = mfcc_data.shape[0]

  #Thêm các đặc trưng vào list chung
  for i in range(len_label):
    labels.append(cname)
    features.append(mfcc_data[i])
print(len(features))
print(len(labels))

3980
3980


In [8]:
#Thực hiện shuffle dữ liệu
c = list(zip(features, labels))
np.random.shuffle(c)
features,labels = zip(*c)

#Chia dữ liệu theo tỷ lệ 70:30
X_train , X_test , y_train , y_test  = train_test_split(features , 
                                                        labels, test_size = 0.3,
                                                        random_state=0)
print(len(X_train))
print(len(y_train))

2786
2786


In [9]:
#Lấy chỉ số tương ứng với label
gmmhmmdict = {}
index = 0
for cname in class_names:
  gmmhmmdict[cname] = index
  index = index +1
gmmhmmdict

{'A': 4,
 'B': 3,
 'ban': 7,
 'len': 0,
 'nhay': 6,
 'phai': 2,
 'trai': 5,
 'xuong': 1}

# Chuẩn bị mô hình
Trong phần này, em thực hiện:
- Thiết lập các tham số đầu vào để huấn luyện mô hình
- Định nghĩa mô hình nhận các tham số đó


In [11]:
#Thiết lập các tham số cần thiết để train GMMHMM
m_num_of_HMMStates = 6  # số trạng thái
m_num_of_mixtures = 2  # số mixtures của mỗi hidden state
m_level = 2 #step size
m_covarianceType = 'diag'  # dạng ma trận covariance 
m_n_iter = 15  # số vòng lặp 

#khởi tạo giá trị cho startprobPrior và transmatPrior
def initLevel(inumstates, m_level):
    startprobPrior = np.zeros(inumstates)
    startprobPrior[0: m_level - 1] = 1/float((m_level - 1))
    transmatPrior = getTransmatPrior(inumstates, m_level)
    return startprobPrior, transmatPrior

#khởi tạo giá trị cho transmatPrior dựa vào step size
def getTransmatPrior(inumstates, m_level):
    transmatPrior = (1 / float(m_level)) * np.eye(inumstates)

    for i in range(inumstates - (m_level - 1)):
        for j in range(m_level - 1):
            transmatPrior[i, i + j + 1] = 1. / m_level

    for i in range(inumstates - m_level + 1, inumstates):
        for j in range(inumstates - i):
            transmatPrior[i, i + j] = 1. / (inumstates - i)

    return transmatPrior

startprobPrior ,transmatPrior = initLevel(m_num_of_HMMStates,m_level)

In [174]:
startprobPrior

array([1., 0., 0., 0., 0., 0.])

In [175]:
transmatPrior

array([[0.5, 0.5, 0. , 0. , 0. , 0. ],
       [0. , 0.5, 0.5, 0. , 0. , 0. ],
       [0. , 0. , 0.5, 0.5, 0. , 0. ],
       [0. , 0. , 0. , 0.5, 0.5, 0. ],
       [0. , 0. , 0. , 0. , 0.5, 0.5],
       [0. , 0. , 0. , 0. , 0. , 1. ]])

In [12]:
#Định nghĩa mô hình
class SpeechModel:
  def __init__(self,Class,label):
    self.traindata = np.zeros((0,39))
    self.Class = Class
    self.label = label
    self.model  = hmm.GMMHMM(n_components = m_num_of_HMMStates, n_mix = m_num_of_mixtures,
                        transmat_prior = transmatPrior, startprob_prior = startprobPrior, 
                                    covariance_type = m_covarianceType, n_iter = m_n_iter, random_state=321)

# Huấn luyện mô hình

Với mỗi label em thực hiện huấn luyện một mô hình hmm tương ứng
---
*Tham số mô hình cần được tuỳ chỉnh để chọn được bộ số tốt nhất. Em cũng đã thử thay đổi nhiều lần các số liệu bên dưới tuy nhiên vẫn chưa tìm được bộ số tốt. Trong nhiều trường hợp, ở bước train em bị báo warning phép chia hoặc ấy log cho 0. Trong tất cả trường hợp em đã thử, ma trận mixture covariance bị suy biến. Do đó kết qủa thực nghiệm còn khá tệ. Hiện tại em chưa tìm được phương pháp điều chỉnh tham số khoa học nào ngoài thử random*

In [177]:
#khởi tạo mảng các mô hình
speechmodels = [None] * len(class_names)
for key in gmmhmmdict:
  speechmodels[gmmhmmdict[key]] = SpeechModel(gmmhmmdict[key],key)

#Phân chia tập dữ liệu huấn luyện theo từng label
for i in range(0,len(X_train)):
  for j in range(0,len(speechmodels)):
    if int(speechmodels[j].Class) == int(gmmhmmdict[y_train[i]]):
      speechmodels[j].traindata = np.concatenate((speechmodels[j].traindata , X_train[i]))

In [178]:
#Huấn luyện từng mô hình tương ứng các label
for speechmodel in speechmodels:
    speechmodel.model.fit(speechmodel.traindata)

In [182]:
#Lưu mô hình ra file
trained_model = {}
for key in gmmhmmdict:
  trained_model[key] = speechmodels[gmmhmmdict[key]]
pickle.dump(trained_model, open(DATA_ROOT_DIR+'model_1.pkl', 'wb'))

# Đánh giá mô hình bằng tập test
Trong phần này em thực hiện đánh giá mô hình sử dụng tập test đã chia bên trên. Nhãn dự đoán là nhãn ứng với mô hình mang lại score lớn nhất

In [16]:
trained_models = pickle.load(open(DATA_ROOT_DIR+'model_1.pkl', 'rb'))

In [22]:
trained_models

{'A': <__main__.SpeechModel at 0x7fcb43c10590>,
 'B': <__main__.SpeechModel at 0x7fcb43c10e10>,
 'ban': <__main__.SpeechModel at 0x7fcb433e2310>,
 'len': <__main__.SpeechModel at 0x7fcb440fc610>,
 'nhay': <__main__.SpeechModel at 0x7fcb433e21d0>,
 'phai': <__main__.SpeechModel at 0x7fcb4a2f8e90>,
 'trai': <__main__.SpeechModel at 0x7fcb433e2250>,
 'xuong': <__main__.SpeechModel at 0x7fcb433e2a10>}

In [179]:
y_pred = []

for i in range(0,len(X_test)):
    scores = []
    for speechmodel in trained_models:
         scores.append(speechmodel.model.score(X_test[i]))
    id  = scores.index(max(scores))
    y_pred.append(trained_models[id].label)
    # print(str(np.round(scores, 3)) + " " + str(max(np.round(scores, 3))) +" "+":"+ speechmodels[id].label)

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mi

In [180]:
from sklearn.metrics import classification_report

#In ra kết quả trên tập test
report = classification_report(y_test, y_pred, target_names=class_names)
print(report)

              precision    recall  f1-score   support

         len       0.23      0.96      0.37       145
       xuong       0.97      0.26      0.41       150
        phai       0.21      0.29      0.25       154
           B       0.58      0.55      0.56       161
           A       1.00      0.02      0.04       174
        trai       0.94      0.11      0.20       141
        nhay       0.47      0.52      0.49       128
         ban       1.00      0.15      0.26       141

    accuracy                           0.35      1194
   macro avg       0.68      0.36      0.32      1194
weighted avg       0.68      0.35      0.32      1194



In [181]:
from sklearn.metrics import accuracy_score
print('accuracy = ',accuracy_score(y_test, y_pred))

accuracy =  0.3500837520938023


# Demo nhận dạng với dữ liệu mới thu

In [29]:
def extract_mfcc_features(file_path,n_mfcc):
    #load file âm thanh
    sound, sr = librosa.load(file_path)

    #trích xuất đặc trưng
    mfcc = librosa.feature.mfcc(y=sound, sr=sr, n_mfcc=n_mfcc,hop_length=512)
    delta_mfcc = librosa.feature.delta(mfcc,width=9)
    delta2_mfcc = librosa.feature.delta(mfcc, order=2)

    #chuẩn hoá
    mfcc_features = np.concatenate((mfcc, delta_mfcc, delta2_mfcc)).T
    min_features = np.min(mfcc_features, axis=0)
    max_features = np.max(mfcc_features, axis=0)
    mfcc_features_nom = (mfcc_features - min_features) / (max_features - min_features)
    
    return mfcc_features_nom

In [32]:
n_mfcc=13
audio_file = "/content/drive/MyDrive/Speech/Data/len.wav"
ipd.Audio(audio_file)
# load audio files with librosa
signal, sr = librosa.load(audio_file)
mfccs_features = extract_mfcc_features(audio_file,n_mfcc)
mfccs_features.shape
# mfccs_features

(30, 39)

In [33]:
scores = []
for speechmodel in trained_models:
  scores.append(trained_models[speechmodel].model.score(mfccs_features))
id = scores.index(max(scores))

def get_key(val):
  for key, value in gmmhmmdict.items():
    if val == value:
      return key

print(get_key(id))

Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance


len
